In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df1 = pd.read_csv ("total.csv")
# df1

In [3]:
# ---------------- 1. 필요한 열만 선택 ----------------
df = df1[["날짜", "호선", "역명", "상하구분", "기온", "시간강수량", "혼잡도"]].copy()

# ---------------- 2. 날짜 관련 파생 컬럼 생성 ----------------
# 날짜 컬럼: yyyymmddhh 형식 → 문자열 길이 체크
df = df[df['날짜'].astype(str).str.len() == 10]

df['년'] = df['날짜'].astype(str).str[0:4].astype(int)
df['월'] = df['날짜'].astype(str).str[4:6].astype(int)
df['일'] = df['날짜'].astype(str).str[6:8].astype(int)
df['시'] = df['날짜'].astype(str).str[8:10].astype(int)


# 계절 함수 정의
def 계절_정의(월):
    if 월 in [3, 4, 5]: return '봄'
    elif 월 in [6, 7, 8]: return '여름'
    elif 월 in [9, 10, 11]: return '가을'
    else: return '겨울'

df['계절'] = df['월'].apply(계절_정의)
df['시간대'] = df['시'].apply(lambda x: f"{x:02d}시")

# 시간대 컬럼 만들기 (예: 07시, 08시)
df['시간대'] = df['시'].apply(lambda x: f"{x:02d}시")

In [4]:
# ---------------- 3. 강수 상태 파생 ----------------
def 비상태_정의(강수량):
    if 강수량 < 1: return '강수량 없음'
    elif 1 <= 강수량 < 3: return '약한 비 or 눈'
    elif 3 <= 강수량 < 15: return '보통 비 or 눈'
    else: return '강한 비 or 눈'

df['시간강수량_상태'] = df['시간강수량'].apply(비상태_정의)

df

,날짜,호선,역명,상하구분,기온,시간강수량,혼잡도,년,월,일,시,계절,시간대,시간강수량_상태
0,2021010100,1,서울역,상선,-9.6,0.0,0,2021,1,1,0,겨울,00시,강수량 없음
1,2021010101,1,서울역,상선,-9.7,0.0,0,2021,1,1,1,겨울,01시,강수량 없음
2,2021010105,1,서울역,상선,-9.3,0.0,1,2021,1,1,5,겨울,05시,강수량 없음
3,2021010106,1,서울역,상선,-9.3,0.0,2,2021,1,1,6,겨울,06시,강수량 없음
4,2021010107,1,서울역,상선,-9.1,0.0,3,2021,1,1,7,겨울,07시,강수량 없음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17412599,2023123119,8,남위례,하선,0.6,0.0,18,2023,12,31,19,겨울,19시,강수량 없음
17412600,2023123120,8,남위례,하선,0.0,0.0,17,2023,12,31,20,겨울,20시,강수량 없음
17412601,2023123121,8,남위례,하선,-0.6,0.0,21,2023,12,31,21,겨울,21시,강수량 없음
17412602,2023123122,8,남위례,하선,-0.8,0.0,18,2023,12,31,22,겨울,22시,강수량 없음


In [10]:
# ---------------- 4. 이상치 및 결측치 처리 ----------------

# (1) 시간강수량 -99 → NaN (이상치 개수)
count_minus_99 = (df['시간강수량'] == -99).sum()
print(f"❗ '-99' 값 개수 (시간강수량): {count_minus_99}")
df['시간강수량'] = df['시간강수량'].replace(-99, np.nan)

# (1-2) 시간강수량 결측치 개수 (NaN 포함)
missing_rain = df['시간강수량'].isna().sum()
print(f"❗ 시간강수량 결측치 개수 (NaN): {missing_rain}")

# (1-3) 시간강수량 결측치 제거
before_rain_nan_drop = len(df)
df = df.dropna(subset=['시간강수량'])
removed_rain_nan = before_rain_nan_drop - len(df)
print(f"🧹 시간강수량 결측치 제거 수: {removed_rain_nan}")

# (2) 기온 이상치 제거 (-30 < 기온 < 50)
before_temp = len(df)
df = df[(df['기온'] > -30) & (df['기온'] < 50)]
removed_temp = before_temp - len(df)
print(f"🌡️ 기온 이상치 제거 수: {removed_temp}")

# (3) 시간강수량 이상치 제거 (0 이상만 허용)
before_rain = len(df)
df = df[df['시간강수량'] >= 0]
removed_rain_outlier = before_rain - len(df)
print(f"🌧️ 시간강수량 이상치(0 미만) 제거 수: {removed_rain_outlier}")

# (4) 혼잡도 결측치 제거
missing_cong = df['혼잡도'].isna().sum()
print(f"❗ 혼잡도 결측치 개수: {missing_cong}")
before_cong = len(df)
df = df.dropna(subset=['혼잡도'])
removed_cong_nan = before_cong - len(df)
print(f"🧹 혼잡도 결측치 제거 수: {removed_cong_nan}")

# (4-2) 혼잡도 이상치 제거 (0 ≤ 혼잡도 ≤ 100)
before_cong_outlier = len(df)
df = df[(df['혼잡도'] >= 0) & (df['혼잡도'] <= 100)]
removed_cong_outlier = before_cong_outlier - len(df)
print(f"🚇 혼잡도 이상치 제거 수: {removed_cong_outlier}")

# (5) 중복 제거
before_dup = len(df)
df = df.drop_duplicates()
removed_dup = before_dup - len(df)
print(f"🔁 중복 제거 수: {removed_dup}")

# 최종 데이터 크기
print(f"\n✅ 최종 데이터 개수: {len(df)}")

❗ '-99' 값 개수 (시간강수량): 0
❗ 시간강수량 결측치 개수 (NaN): 0
🧹 시간강수량 결측치 제거 수: 0
🌡️ 기온 이상치 제거 수: 0
🌧️ 시간강수량 이상치(0 미만) 제거 수: 0
❗ 혼잡도 결측치 개수: 0
🧹 혼잡도 결측치 제거 수: 0
🚇 혼잡도 이상치 제거 수: 0
🔁 중복 제거 수: 0

✅ 최종 데이터 개수: 16128804


In [11]:
# ---------------- 5. 통계 요약 ----------------
print(f"\n📉 최종적으로 줄어든 데이터 개수: {len(df1) - len(df)}")
print(f"🎯 이상치 제거 후 남은 데이터 개수: {len(df)}")

print("\n📊 '기온' 범위 확인:")
print(f"최소값: {df['기온'].min()} / 최대값: {df['기온'].max()}")

print("\n📊 '시간강수량' 범위 확인:")
print(f"최소값: {df['시간강수량'].min()} / 최대값: {df['시간강수량'].max()}")

print("\n📊 '혼잡도' 범위 확인:")
print(f"최소값: {df['혼잡도'].min()} / 최대값: {df['혼잡도'].max()}")


📉 최종적으로 줄어든 데이터 개수: 1283800
🎯 이상치 제거 후 남은 데이터 개수: 16128804

📊 '기온' 범위 확인:
최소값: -25.2 / 최대값: 39.6

📊 '시간강수량' 범위 확인:
최소값: 0.0 / 최대값: 136.5

📊 '혼잡도' 범위 확인:
최소값: 0 / 최대값: 100


In [12]:
# ---------------- 6. 혼잡도 분석 ----------------
grouped = df.groupby(['계절', '시간대', '시간강수량_상태'])['혼잡도'].mean().reset_index()
print("\n📈 계절 + 시간대 + 비상태별 혼잡도 평균")
print(grouped.head())


📈 계절 + 시간대 + 비상태별 혼잡도 평균
   계절  시간대   시간강수량_상태       혼잡도
0  가을  00시     강수량 없음  3.716313
1  가을  00시  강한 비 or 눈  1.618307
2  가을  00시  보통 비 or 눈  2.689775
3  가을  00시  약한 비 or 눈  2.160403
4  가을  01시     강수량 없음  0.009814


In [13]:
# 결과 CSV로 저장
df.to_csv("total_지하철_혼잡도_계절별_시간대별_강수상태별_분석.csv", encoding='utf-8-sig', index=False)